In [2]:
import importlib.util
import sys
from chat_manager import ChatManager
from utils.token_counter import count_tokens
import logging

In [3]:
def load_conversation(file_path):
    print(f"Loading conversation from file: {file_path}")
    spec = importlib.util.spec_from_file_location("sample_conversation", file_path)
    module = importlib.util.module_from_spec(spec)
    sys.modules["sample_conversation"] = module
    spec.loader.exec_module(module)
    print(f"Loaded conversation with {len(module.conversation)} messages")
    return module.conversation



In [4]:

conversation = load_conversation('data/sample_conversation.py')
chat_manager = ChatManager()
approaches = chat_manager.approaches.keys()
results = {approach: [] for approach in approaches}

Loading conversation from file: data/sample_conversation.py
Loaded conversation with 38 messages


In [5]:
chat_manager.set_approach('advanced')

In [6]:
approach = 'advanced'
cumulative_tokens = 0

In [9]:
for i, message in enumerate(conversation):
    user_id = f"user_{approach}"
    if message['role'] == 'user':
        context = chat_manager.get_context(user_id)
        prompt = f"Context: {context}\n\nUser: {message['content']}\n\nAssistant:"
        tokens = count_tokens(prompt)
        cumulative_tokens += tokens
        results[approach].append((i + 1, cumulative_tokens))
        print(f"Approach: {approach}, Message: {i+1}, Cumulative Tokens: {cumulative_tokens}")
    chat_manager.handle_new_message(user_id, message)

: 

In [9]:
for i, message in enumerate(conversation):
    print(message)
    user_id = f"user_{approach}"
    if message['role'] == 'user':
        context = chat_manager.get_context(user_id)
        print(context)
    break

{'role': 'user', 'content': 'Can you tell me what AWS is all about?'}
[]


In [1]:
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
from typing import List, Dict, Tuple
import json

class AdvancedContextManager:
    def __init__(self, model_name: str = "sentence-transformers/all-MiniLM-L6-v2"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.short_term_memory: List[Dict] = []
        self.long_term_memory: List[Dict] = []
        self.index = faiss.IndexFlatL2(384)  # 384 is the embedding dimension for the chosen model
        self.max_short_term_tokens = 800
        self.max_context_tokens = 2000

    def add_message(self, user_id: str, message: Dict):
        # Add to short-term memory
        print(message)
        self.short_term_memory.append(message)
        print("short_term_memory", self.short_term_memory)
        self._trim_short_term_memory()

        # Add to long-term memory
        embedding = self._get_embedding(message['content'])
        print("embedding", embedding)
        self.long_term_memory.append({**message, 'embedding': embedding})
        self.index.add(np.array([embedding]))

    def get_context(self, user_id: str) -> str:
        context = self._get_short_term_context()
        long_term_context = self._get_long_term_context(user_id)
        return json.dumps(context + long_term_context)

    def _get_embedding(self, text: str) -> np.ndarray:
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        with torch.no_grad():
            outputs = self.model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

    def _trim_short_term_memory(self):
        while len(self.short_term_memory) > 4 or self._count_tokens(self.short_term_memory) > self.max_short_term_tokens:
            self.short_term_memory.pop(0)

    def _get_short_term_context(self) -> List[Dict]:
        return self.short_term_memory

    def _get_long_term_context(self, user_id: str) -> List[Dict]:
        if not self.long_term_memory:
            return []

        query = self.short_term_memory[-1]['content'] if self.short_term_memory else ""
        query_embedding = self._get_embedding(query)

        k = min(5, len(self.long_term_memory))  # Get top 5 or all if less than 5
        D, I = self.index.search(np.array([query_embedding]), k)

        results = [self.long_term_memory[i] for i in I[0]]
        results.sort(key=lambda x: x['timestamp'], reverse=True)  # Sort by recency

        return results[:2]  # Return top 2 most relevant and recent

    def _count_tokens(self, messages: List[Dict]) -> int:
        return sum(len(self.tokenizer.encode(msg['content'])) for msg in messages)

    def get_internal_state(self, user_id: str) -> Dict:
        return {
            "short_term_memory": self.short_term_memory,
            "long_term_memory_size": len(self.long_term_memory),
            "current_context_tokens": self._count_tokens(self.short_term_memory)
        }

/opt/anaconda3/envs/chat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import sys
sys.path.append('..')  # Add the parent directory to the Python path

# from advanced_context_manager import AdvancedContextManager
import json
from datetime import datetime, timedelta

context_manager = AdvancedContextManager()

In [4]:
from transformers import AutoTokenizer, AutoModel

/opt/anaconda3/envs/chat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
AutoTokenizer.from_pretrained(model_name)


BertTokenizerFast(name_or_path='sentence-transformers/all-MiniLM-L6-v2', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [2]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

NameError: name 'AutoTokenizer' is not defined

In [6]:
messages[0]['content']

{'role': 'user',
 'content': 'Hello, how are you?',
 'timestamp': '2024-10-11T01:12:48.438125'}

In [7]:
embedding = context_manager._get_embedding(messages[0]['content'])

: 

In [5]:
def create_message(role, content, timestamp=None):
    if timestamp is None:
        timestamp = datetime.now().isoformat()
    return {"role": role, "content": content, "timestamp": timestamp}

messages = [
    create_message("user", "Hello, how are you?"),
    create_message("assistant", "I'm doing well, thank you for asking. How can I assist you today?"),
    create_message("user", "Can you tell me about the weather today?"),
    create_message("assistant", "I'm sorry, but I don't have access to real-time weather information. Is there something else I can help you with?"),
    create_message("user", "Okay, can you explain the concept of machine learning?"),
    create_message("assistant", "Certainly! Machine learning is a subset of artificial intelligence..."),
    # Add more messages as needed
]

In [4]:
for i, message in enumerate(messages):
    context_manager.add_message("test_user", message)
    print(f"After message {i+1}:")
    print(json.loads(context_manager.get_context("test_user")))
    print("\nInternal state:")
    print(context_manager.get_internal_state("test_user"))
    print("\n" + "="*50 + "\n")
    break

: 